# Generating statistics for subset of Wikidata

This notebook illustrates how to generate statistics for a subset of Wikidata. \
We use as an example https://www.wikidata.org/wiki/Q11173 (chemical compound).

Example Dataset wikidata subset: https://drive.google.com/drive/u/1/folders/1KjNwV5M2G3JzCrPgqk_TSx8wTE49O2Sx \
Example Dataset statistics: https://drive.google.com/drive/u/1/folders/1_4Mxd0MAo0l9aR3aInv0YMTJrtneh7HW 

### Example Invocation command

    papermill /Users/shashanksaurabh/Desktop/MS/ISI/isi/kgtk_shashank73744/kgtk/examples/Example_9-Wikidata_Subset_Statistics_.ipynb \
    /Users/shashanksaurabh/Desktop/MS/ISI/isi/kgtk_shashank73744/kgtk/examples/Example_9_output.ipynb \
    -p wikidata_home '/Users/shashanksaurabh/Desktop/Data_isi' \
    -p wikidata_parts_folder '/Users/shashanksaurabh/Desktop/Data_isi/Chemical' \
    -p cache_folder '/Users/shashanksaurabh/Desktop/Data_isi/Temp' \
    -p output_folder '/Users/shashanksaurabh/Desktop/Data_isi/output' \
    -p delete_database 'yes' \
    -p K \"10\" \
    -p subset_name 'Q11173'


# Naming Convention:

## [subset_name].[section].[brief_description].tsv

## subset_name 
It is the qnode corresponding to the  wikidata subset, for example if the wikidata subset refers to the chemical compounds then “subset_name” is ”Q11173”

## section 
1. It is the section corresponding to the “Knowledge Graph Statistics” described above. For example if its class summary then it would be ‘1’. 

2. If there are more than one subsection for the main section then the subsection would be added after a dot(‘.’). For example ‘2.2’ corresponds to the ‘item properties’ under ‘Properties’. 

3. If there are more than one subsection to the parent subsection then the subsections would be recursively added after a dot(‘.’). For example while calculating the statistics of string for the section 2.6, it would be named 2.6.1

## brief_description:
1. Its a brief description just after section 
2. For class summary it’s ‘class_summary’
3. For ‘examples’ it’s the qnode for which 3 examples are shown
4. Its ‘property_summary_[type]’ for each of the data types in section 2.1.
5. Its ‘item_properties’ for section 2.3
6. Its ‘time_properties’ for section 2.4
7. Its ‘quantity_properties’ for section 2.5
8. Its ‘distincs_values_[type]’ for each of the type for section 2.6
9. Its ‘geo_coordinate_top_m’ for 2.7
10. Its ‘geo_shape_top_m’ for for 2.8


In [70]:
wikidata_home = "/Users/shashanksaurabh/Desktop/Data_isi"
# path to folder which contains all files corresponding to the wikidata subset. 
#(For more information on wikidata subset please check Example 8)
wikidata_parts_folder = "/Users/shashanksaurabh/Desktop/Data_isi/Chemical"
# The notebook creates a cache, which stores in the cache_folder. The cache can be deleted after the execution.
cache_folder = "/Users/shashanksaurabh/Desktop/Data_isi/Temp"
# path to the folder where the output (here statistics) would be stored
output_folder = "/Users/shashanksaurabh/Desktop/Data_isi/output"
# delete_database = "yes"
# The statistics also uses the consolidated kgtk file with all the properties and items. This is the path to consolidated file
# In each of statistics top K results are chosen. In the following examples this has been implemented using the --limit attribute.
K = "10"
subset_name = "Q11173"

In [71]:
import io
import os
import subprocess
import sys

import numpy as np
import pandas as pd

from IPython.display import display, HTML

# import altair as alt
# alt.renderers.enable('altair_viewer')

# from IPython.display import display, HTML, Image
# from pandas_profiling import ProfileReport

### Set up environment variables and folders that we need

In [72]:
# path to folder which contains all files corresponding to the wikidata subset. 
#(For more information on wikidata subset please check Example 8)
os.environ['WIKIDATA_PARTS'] = wikidata_parts_folder
# path to the folder where the output (here statistics) would be stored
os.environ['OUTPUT_FOLDER'] = output_folder
# kgtk command to run
os.environ['kgtk'] = "kgtk"
os.environ['kgtk'] = "time kgtk --debug"
# absolute path of the db
os.environ['STORE'] = "{}/wikidata.sqlite3.db".format(cache_folder)
os.environ['K'] = K
os.environ['label'] = subset_name + ".label.en.tsv.gz"
# file name corresponding to different part of the subsets.
os.environ['subset_name']  = subset_name
os.environ['external_id']  = subset_name + ".part.external-id.tsv.gz"
os.environ['time']  = subset_name +  ".part.time.tsv.gz "
os.environ['wikibase_item']  = subset_name + ".part.wikibase-item.tsv.gz"
os.environ['quantity']  = subset_name +  ".part.quantity.tsv.gz"
os.environ['statistics']  = subset_name + ".statistics.tsv.gz"
os.environ['wikibase_form']  = subset_name + ".part.wikibase-form.tsv.gz"
os.environ['monolingualtext']  = subset_name + ".part.monolingualtext.tsv.gz"
os.environ['math']  = subset_name + ".part.math.tsv.gz"
os.environ['commonsMedia']  = subset_name + ".part.commonsMedia.tsv.gz"
os.environ['globe_coordinate']  = subset_name + ".part.globe-coordinate.tsv.gz"
os.environ['musical_notation']  = subset_name + ".part.musical-notation.tsv.gz"
os.environ['geo_shape']  = subset_name + ".part.geo-shape.tsv.gz"
os.environ['url']  = subset_name + ".part.url.tsv.gz"
os.environ['string']  = subset_name + ".part.string.tsv.gz"
# Output file for statistics 1.1
os.environ['class_summary']  = subset_name + ".1.class_summary.tsv"
# Output files for statistics 2.1
os.environ['property_summary']  = subset_name + ".2.1.property_summary.tsv"
# Output files for statistics 2.2
os.environ['property_summary_external_id']  = subset_name + ".2.2.1property_summary_external_id.tsv"
os.environ['property_summary_time']  = subset_name + ".2.2.2.property_summary_time.tsv"
os.environ['property_summary_wikibase_item']  = subset_name + ".2.2.3.property_summary_wikibase_item.tsv"
os.environ['property_summary_quantity']  = subset_name + ".2.2.4.property_summary_quantity.tsv"
os.environ['property_summary_wikibase_form']  = subset_name + ".2.2.5.property_summary_wikibase_form.tsv"
os.environ['property_summary_monolingualtext']  = subset_name + ".2.2.6.property_summary_monolingualtext.tsv"
os.environ['property_summary_math']  = subset_name + ".2.2.7property_summary_math.tsv"
os.environ['property_summary_commonsMedia']  = subset_name + ".2.2.8.property_summary_commonsMedia.tsv"
os.environ['property_summary_globe_coordinate']  = subset_name + ".2.2.9.property_summary_globe_coordinate.tsv"
os.environ['property_summary_musical_notation']  = subset_name + ".2.2.10.property_summary_musical_notation.tsv"
os.environ['property_summary_geo_shape']  = subset_name + ".2.2.11.property_summary_geo_shape.tsv"
os.environ['property_summary_url']  = subset_name + ".2.2.12.property_summary_url.tsv"
os.environ['property_summary_string']  = subset_name + ".2.2.13.property_summary_string.tsv"
# Output files for statistics 2.3
os.environ['item_properties']  = subset_name + ".2.3.item_properties.tsv"# Output files for statistics 3.1
# Output files for statistics 2.4
os.environ['time_properties']  = subset_name + ".2.4.1.time_properties.tsv"
os.environ['time_properties_count']  = subset_name + ".2.4.2.time_properties.tsv"
# Output files for statistics 2.5
os.environ['quantity_properties']  = subset_name + ".2.5.quantity_properties.tsv"
# Output files for statistics 2.6
os.environ['distinct_values_math']  = subset_name + ".2.6.1.distinct_values_math.tsv"
os.environ['distinct_values_string']  = subset_name + ".2.6.2.distinct_values_string.tsv"
os.environ['distinct_values_external_id']  = subset_name + ".2.6.3.distinct_values_external_id.tsv"
os.environ['distinct_values_monolingual_text']  = subset_name + ".2.6.4.distinct_values_monolingual_text.tsv"
os.environ['distinct_values_musical_notation']  = subset_name + ".2.6.5.distinct_values_musical_notation.tsv"
# Output files for statistics 2.7
os.environ['globe_coordinate_top_m']  = subset_name + ".2.7.globe_coordinate_top_m.tsv"
# Output files for statistics 2.8
os.environ['geo_shape_top_m']  = subset_name + ".2.8.geo_shape_top_m.tsv"

In [73]:
def run_command(cmd, substitution_dictionary = {}):
    """Run a templetized command."""
    for k, v in substitution_dictionary.items():
        cmd = cmd.replace(k, v)
    
    print(cmd)
    output = subprocess.run([cmd], shell=True, universal_newlines=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    print(output.stdout)
    print(output.stderr)
    #print(output.returncode)

In [74]:
def find_statistics(df,column):
    n = df[column].count()
    print("Statistics for : "+column)
    unique = df[column].nunique()
    unique_percent = round((unique/n)*100,2)
    print("Distinct : "+str(unique))
    print("Distinct(%) :"+str(unique_percent)+"%")
    missing = df[column].isnull().sum()
    missing_percent = round((missing/n)*100,2)
    print("Missing : "+str(missing))
    print("Missing(%) : "+str(missing_percent)+"%")
    infinite = df[column].isna().sum()
    infinite_percent = round((infinite/n)*100,2)
    print("Infinite : "+str(infinite))
    print("Infinite(%) : "+str(infinite_percent)+"%")
    mean = df[column].mean()
    print("Mean : "+str(mean))
    max_ = df[column].max()
    print("Max : "+str(max_))
    min_ = df[column].min()
    print("Min : "+str(min_))
    zeros_ = (df[column]==0).sum()
    zeros_percent = round((zeros_/n)*100,2)
    print("Zeros : "+str(zeros_))
    print("Zeros(%) : "+str(unique_percent)+"%")
    

# 1. Classes
   ## 1.1 Class summary
   ### List of top K classes based on number of instances (Done)
   ### class -- is the qnode corresponding to the class 
   ### pnode -- is the property by which the class is linked
   ### name -- is the label for the class
   ### number of instances -- is the number of instances of the class in wikidata subset
   ### pagerank -- is the page rank of class


In [75]:
cmd = "$kgtk query -i $WIKIDATA_PARTS/$wikibase_item -i $WIKIDATA_PARTS/$label -i $WIKIDATA_PARTS/$statistics --graph-cache $STORE \
-o $OUTPUT_FOLDER/$class_summary \
--match 'item: (n1)-[l{label:llab}]->(n2), label: (n2)-[:label]->(label),statistics:(n2)-[:vertex_pagerank]->(pagerank) ' \
--return 'distinct n2 as class, label as name, count(n2) as value_count, pagerank as pagerank' \
--where '(label.kgtk_lqstring_lang_suffix = \"en\") AND (llab IN [\"P31\" , \"P279\"]) AND (n2 != \"__subset_name\") ' \
--order-by 'count(n2) desc'"

run_command(cmd, {"__subset_name": subset_name})

$kgtk query -i $WIKIDATA_PARTS/$wikibase_item -i $WIKIDATA_PARTS/$label -i $WIKIDATA_PARTS/$statistics --graph-cache $STORE -o $OUTPUT_FOLDER/$class_summary --match 'item: (n1)-[l{label:llab}]->(n2), label: (n2)-[:label]->(label),statistics:(n2)-[:vertex_pagerank]->(pagerank) ' --return 'distinct n2 as class, label as name, count(n2) as value_count, pagerank as pagerank' --where '(label.kgtk_lqstring_lang_suffix = "en") AND (llab IN ["P31" , "P279"]) AND (n2 != "Q11173") ' --order-by 'count(n2) desc'

[2020-10-20 13:01:31 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_20_c2."node1" "class", graph_20_c2."node2" "name", count(graph_20_c2."node1") "value_count", graph_21_c3."node2" "pagerank"
     FROM graph_19 AS graph_19_c1, graph_20 AS graph_20_c2, graph_21 AS graph_21_c3
     WHERE graph_19_c1."label"=graph_19_c1."label"
     AND graph_20_c2."label"=?
     AND graph_21_c3."label"=?
     AND graph_19_c1."node2"=graph_20_c2."node1"
     AN

In [76]:
df_class_summary = pd.read_csv(os.path.join(os.getenv('OUTPUT_FOLDER'),os.getenv('class_summary')),delimiter='\t')
df_class_summary.head()

,class,name,value_count,pagerank
0,Q12140,'medication'@en,5044,0.000111
1,Q63436503,'diacylglycerophosphocholine'@en,970,0.000025
2,Q187661,'carcinogen'@en,958,0.000027
3,Q63446172,'wax monoester'@en,820,0.000021
4,Q11367,'lipid'@en,704,0.000022


## Statistics for 'value_count' column

In [77]:
find_statistics(df_class_summary,"value_count")

Statistics for : value_count
Distinct : 106
Distinct(%) :4.63%
Missing : 0
Missing(%) : 0.0%
Infinite : 0
Infinite(%) : 0.0%
Mean : 17.351374945438675
Max : 5044
Min : 2
Zeros : 0
Zeros(%) : 4.63%


## Statistics for 'pagerank' column

In [27]:
find_statistics(df_class_summary,"pagerank")

Statistics for : pagerank
Distinct : 2288
Distinct(%) :99.87%
Missing : 0
Missing(%) : 0.0%
Infinite : 0
Infinite(%) : 0.0%
Mean : 1.5944703872654804e-06
Max : 0.00012463927532536587
Min : 9.33084945063929e-08
Zeros : 0
Zeros(%) : 99.87%


 ### Example wikibase item for each of the class.
 ### qnode -- is the qnode of the example wikibase item
 ### name -- is the label of qnode
 ### pagerank -- is the page rank of the qnode


In [31]:
try:
    df = pd.read_csv(os.path.join(os.getenv('OUTPUT_FOLDER'),os.getenv('class_summary')),delimiter='\t')
    cmd = "$kgtk query -i $WIKIDATA_PARTS/$wikibase_item  -i $WIKIDATA_PARTS/$label -i $WIKIDATA_PARTS/$statistics --graph-cache $STORE \
    -o $OUTPUT_FOLDER/__output_file \
    --match 'item: (n1)-[l]->(n2:__class), label: (n1)-[:label]->(label), statistics:(n1)-[:vertex_pagerank]->(pagerank)' \
    --return 'n1 as qnode, label as name, pagerank as pagerank' \
    --where 'label.kgtk_lqstring_lang_suffix = \"en\"' \
    --order-by pagerank \
    --limit 3"
    for index, ele in df.iterrows():
        if index>10:
            break
        if ele["class"] ==  subset_name:
            continue
        print("Examples for ", ele["class"]," class")
        output_file = subset_name + ".1." + ele["class"]+".examples"+".tsv"
        run_command(cmd, {"__output_file": output_file,"__class": ele["class"]})
except Exception as e:
    print(e)

Examples for  Q12140  class
$kgtk query -i $WIKIDATA_PARTS/$wikibase_item  -i $WIKIDATA_PARTS/$label -i $WIKIDATA_PARTS/$statistics --graph-cache $STORE     -o $OUTPUT_FOLDER/Q11173.1.Q12140.examples.tsv     --match 'item: (n1)-[l]->(n2:Q12140), label: (n1)-[:label]->(label), statistics:(n1)-[:vertex_pagerank]->(pagerank)'     --return 'n1 as qnode, label as name, pagerank as pagerank'     --where 'label.kgtk_lqstring_lang_suffix = "en"'     --order-by pagerank     --limit 3

[2020-10-20 11:35:00 query]: SQL Translation:
---------------------------------------------
  SELECT graph_19_c1."node1" "qnode", graph_20_c2."node2" "name", graph_21_c3."node2" "pagerank"
     FROM graph_19 AS graph_19_c1, graph_20 AS graph_20_c2, graph_21 AS graph_21_c3
     WHERE graph_19_c1."node2"=?
     AND graph_20_c2."label"=?
     AND graph_21_c3."label"=?
     AND graph_19_c1."node1"=graph_20_c2."node1"
     AND graph_20_c2."node1"=graph_21_c3."node1"
     AND (kgtk_lqstring_lang_suffix(graph_20_c2."node


[2020-10-20 11:35:06 query]: SQL Translation:
---------------------------------------------
  SELECT graph_20_c2."node1" "qnode", graph_20_c2."node2" "name", graph_21_c3."node2" "pagerank"
     FROM graph_19 AS graph_19_c1, graph_20 AS graph_20_c2, graph_21 AS graph_21_c3
     WHERE graph_19_c1."node2"=?
     AND graph_20_c2."label"=?
     AND graph_21_c3."label"=?
     AND graph_19_c1."node1"=graph_20_c2."node1"
     AND graph_19_c1."node1"=graph_21_c3."node1"
     AND (kgtk_lqstring_lang_suffix(graph_20_c2."node2") = ?)
     ORDER BY graph_21_c3."node2" ASC
     LIMIT ?
  PARAS: ['Q193430', 'label', 'vertex_pagerank', 'en', 3]
---------------------------------------------
        0.68 real         0.50 user         0.13 sys

Examples for  Q422248  class
$kgtk query -i $WIKIDATA_PARTS/$wikibase_item  -i $WIKIDATA_PARTS/$label -i $WIKIDATA_PARTS/$statistics --graph-cache $STORE     -o $OUTPUT_FOLDER/Q11173.1.Q422248.examples.tsv     --match 'item: (n1)-[l]->(n2:Q422248), label: (n1)-[

In [33]:
df_class_example = []
try:
    df = pd.read_csv(os.path.join(os.getenv('OUTPUT_FOLDER'),os.getenv('class_summary')),delimiter='\t')
    for index, ele in df.iterrows():
        if index>10:
            break
        if ele["class"] ==  subset_name:
            continue
        output_file = subset_name + ".1." + ele["class"]+".examples"+".tsv"
        df_class_example.append(pd.read_csv(os.path.join(os.getenv('OUTPUT_FOLDER'),output_file),delimiter='\t'))
    for i in range(len(df_class_example)):
        print("Example for "+ele["class"])
        display(df_class_example[i])
        print("-------------------------")
except Exception as e:
    print(e)

Example for Q134856


,qnode,name,pagerank
0,Q21011228,'dulaglutide'@en,0.000001
1,Q21011228,'dulaglutide'@en,0.000001
2,Q27270940,'tezacaftor'@en,0.000001


-------------------------
Example for Q134856


,qnode,name,pagerank
0,Q27105002,"'1-hexadecanoyl-2-(9Z,12Z-octadecadienoyl)-sn-...",0.000001
1,Q27145082,'1-stearoyl-2-palmitoyl-sn-glycero-3-phosphoch...,0.000001
2,Q27145169,'1-palmitoyl-2-acetyl-sn-glycero-3-phosphochol...,0.000001


-------------------------
Example for Q134856


...

-------------------------
Example for Q134856


,qnode,name,pagerank
0,Q27257146,'(3E)-3-octenoic acid'@en,0.000001
1,Q2707986,'paullinic acid'@en,0.000001
2,Q27277849,'4-ethyloctanoic acid'@en,0.000001


-------------------------
Example for Q134856


,qnode,name,pagerank
0,Q27098079,'icosanoyl-CoA'@en,0.000001
1,Q27098086,'pentanoyl-CoA'@en,0.000001
2,Q27117097,'heptanoyl-CoA'@en,0.000001


-------------------------


# 2. Property

# 2.1 First a summary by property type
## For each type
### pnode, label, count of items with property (see example below)
### Example (show an instance or two of this property)



In [34]:
types = [
    ("time","time","property_summary_time"),
    ("wikibase_item","wikibase_item","property_summary_wikibase_item"),
    ("math","math","property_summary_math"),
    ("wikibase_form","wikibase-form","property_summary_wikibase_form"),
    ("quantity","quantity","property_summary_quantity"),
    ("string","string","property_summary_string"),
    ("external_id","external-id","property_summary_external_id"),
    ("commonsMedia","commonsMedia","property_summary_commonsMedia"),
    ("globe_coordinate","globe-coordinate","property_summary_globe_coordinate"),
    ("monolingualtext","monolingualtext","property_summary_monolingualtext"),
    ("musical_notation","musical-notation","property_summary_musical_notation"),
    ("geo_shape","geo-shape","property_summary_geo_shape"),
    ("url","url","property_summary_url"),
]

cmd = "$kgtk query  -i $WIKIDATA_PARTS/$TYPE_FILE -i $WIKIDATA_PARTS/$label --graph-cache $STORE \
-o $OUTPUT_FOLDER/$output_file \
--match 'part: (n1)-[l{label: llab}]->(n2), label: (llab)-[:label]->(label)' \
--return 'distinct llab as property, count(llab) as number_of_statements, count(n2) as number_of_nodes_having_the_property, label as `label`' \
--where 'label.kgtk_lqstring_lang_suffix = \"en\"' \
--order-by 'count(llab) desc, count(n2) desc'"

for type,name, output_file in types:
    run_command(cmd, {"TYPE_FILE": type,"output_file":output_file})

$kgtk query  -i $WIKIDATA_PARTS/$time -i $WIKIDATA_PARTS/$label --graph-cache $STORE -o $OUTPUT_FOLDER/$property_summary_time --match 'part: (n1)-[l{label: llab}]->(n2), label: (llab)-[:label]->(label)' --return 'distinct llab as property, count(llab) as number_of_statements, count(n2) as number_of_nodes_having_the_property, label as `label`' --where 'label.kgtk_lqstring_lang_suffix = "en"' --order-by 'count(llab) desc, count(n2) desc'

[2020-10-20 11:43:36 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_17_c1."label" "property", count(graph_17_c1."label") "number_of_statements", count(graph_17_c1."node2") "number_of_nodes_having_the_property", graph_20_c2."node2" "label"
     FROM graph_17 AS graph_17_c1, graph_20 AS graph_20_c2
     WHERE graph_17_c1."label"=graph_17_c1."label"
     AND graph_20_c2."label"=?
     AND graph_17_c1."label"=graph_20_c2."node1"
     AND (kgtk_lqstring_lang_suffix(graph_20_c2."node2") = ?)
     GROUP BY proper


[2020-10-20 11:44:26 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_20_c2."node1" "property", count(graph_20_c2."node1") "number_of_statements", count(graph_25_c1."node2") "number_of_nodes_having_the_property", graph_20_c2."node2" "label"
     FROM graph_20 AS graph_20_c2, graph_25 AS graph_25_c1
     WHERE graph_20_c2."label"=?
     AND graph_25_c1."label"=graph_20_c2."node1"
     AND graph_20_c2."node1"=graph_25_c1."label"
     AND (kgtk_lqstring_lang_suffix(graph_20_c2."node2") = ?)
     GROUP BY property
     ORDER BY count(graph_20_c2."node1") DESC, count(graph_25_c1."node2") DESC
  PARAS: ['label', 'en']
---------------------------------------------
        1.54 real         0.80 user         0.21 sys

$kgtk query  -i $WIKIDATA_PARTS/$globe_coordinate -i $WIKIDATA_PARTS/$label --graph-cache $STORE -o $OUTPUT_FOLDER/$property_summary_globe_coordinate --match 'part: (n1)-[l{label: llab}]->(n2), label: (llab)-[:label]->(label)' --retu

In [39]:
df_property_summary = []
types = [
    ("time","time","property_summary_time"),
    ("wikibase_item","wikibase_item","property_summary_wikibase_item"),
    ("math","math","property_summary_math"),
    ("wikibase_form","wikibase-form","property_summary_wikibase_form"),
    ("quantity","quantity","property_summary_quantity"),
    ("string","string","property_summary_string"),
    ("external_id","external-id","property_summary_external_id"),
    ("commonsMedia","commonsMedia","property_summary_commonsMedia"),
    ("globe_coordinate","globe-coordinate","property_summary_globe_coordinate"),
    ("monolingualtext","monolingualtext","property_summary_monolingualtext"),
    ("musical_notation","musical-notation","property_summary_musical_notation"),
    ("geo_shape","geo-shape","property_summary_geo_shape"),
    ("url","url","property_summary_url"),
]
for type,name, output_file in types:
    try:
        temp = pd.read_csv(os.path.join(os.getenv('OUTPUT_FOLDER'),os.getenv(output_file)),delimiter='\t')
    except Exception as e:
        continue
    df_property_summary.append(temp)
    if  len(df_property_summary[-1])>0:
        display(df_property_summary[-1].head())
        df_property_summary[-1].describe()

,property,number_of_statements,number_of_nodes_having_the_property,label
0,P575,20,20,'time of discovery or invention'@en
1,P2669,1,1,'discontinued date'@en
2,P571,1,1,'inception'@en
3,P729,1,1,'service entry'@en
4,P730,1,1,'service retirement'@en


,property,number_of_statements,number_of_nodes_having_the_property,label
0,P972,49434,49434,'catalog'@en
1,P527,15973,15973,'has part'@en
2,P361,9430,9430,'part of'@en
3,P2175,6121,6121,'medical condition treated'@en
4,P3780,4178,4178,'active ingredient in'@en


,property,number_of_statements,number_of_nodes_having_the_property,label
0,P2067,146546,146546,'mass'@en
1,P2101,9494,9494,'melting point'@en
2,P2054,1113,1113,'density'@en
3,P4250,908,908,'defined daily dose'@en
4,P2102,873,873,'boiling point'@en


,property,number_of_statements,number_of_nodes_having_the_property,label
0,P274,344854,344854,'chemical formula'@en
1,P233,199853,199853,'canonical SMILES'@en
2,P2017,141589,141589,'isomeric SMILES'@en
3,P373,3463,3463,'Commons category'@en
4,P1931,615,615,'NIOSH Pocket Guide ID'@en


,property,number_of_statements,number_of_nodes_having_the_property,label
0,P235,997389,997389,'InChIKey'@en
1,P234,990406,990406,'InChI'@en
2,P231,927660,927660,'CAS Registry Number'@en
3,P3117,848585,848585,'DSSTox substance ID'@en
4,P662,250629,250629,'PubChem CID'@en


,property,number_of_statements,number_of_nodes_having_the_property,label
0,P117,12520,12520,'chemical structure'@en
1,P18,1992,1992,'image'@en
2,P8224,20,20,'molecular model'@en
3,P443,17,17,'pronunciation audio'@en
4,P989,6,6,'spoken text audio'@en


,property,number_of_statements,number_of_nodes_having_the_property,label
0,P625,1,1,'coordinate location'@en


,property,number_of_statements,number_of_nodes_having_the_property,label
0,P2275,2414,2414,'World Health Organisation International Nonpr...
1,P1705,13,13,'native label'@en
2,P1813,9,9,'short name'@en
3,P2561,4,4,'name'@en
4,P1448,1,1,'official name'@en


,property,number_of_statements,number_of_nodes_having_the_property,label
0,P2888,41,41,'exact match'@en
1,P856,26,26,'official website'@en
2,P6363,4,4,'WordLift URL'@en
3,P1482,1,1,'Stack Exchange tag'@en
4,P1709,1,1,'equivalent class'@en


# 2.2 Divided by data type of property


## For each data type select top K properties based on number of statements we are finding the units and the value_counts of the unit

## Header
### units -- The qnode corresponding to the unit
### value_Count -- number of instances of the unit

In [40]:
df_quantity_data_type = []
try:
    df = pd.read_csv(os.path.join(os.getenv('OUTPUT_FOLDER'),os.getenv('property_summary_quantity')),delimiter='\t')
    cmd = "$kgtk query -i $WIKIDATA_PARTS/$quantity --graph-cache $STORE \
            -o $OUTPUT_FOLDER/__output \
            --match  '(n1)-[r:__property]->(v)' \
            --return 'distinct kgtk_quantity_wd_units(v) as units, count(v) as value_count' \
            --where 'kgtk_quantity(v)' \
            --order-by 'count(v) desc' "
    for index, ele in df.iterrows():
        output_file = subset_name + ".2.2.5." + ele["property"]+".data_types"+".tsv"
        run_command(cmd, {"__property": ele["property"],"__output": output_file})
        temp = pd.read_csv(os.path.join(os.getenv('OUTPUT_FOLDER'),output_file),delimiter='\t')
        df_quantity_data_type.append(temp)
except Exception as e:
    print(e)

$kgtk query -i $WIKIDATA_PARTS/$quantity --graph-cache $STORE             -o $OUTPUT_FOLDER/Q11173.2.2.5.P2067.data_types.tsv             --match  '(n1)-[r:P2067]->(v)'             --return 'distinct kgtk_quantity_wd_units(v) as units, count(v) as value_count'             --where 'kgtk_quantity(v)'             --order-by 'count(v) desc' 

[2020-10-20 11:50:35 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT kgtk_quantity_wd_units(graph_16_c1."node2") "units", count(graph_16_c1."node2") "value_count"
     FROM graph_16 AS graph_16_c1
     WHERE graph_16_c1."label"=?
     AND kgtk_quantity(graph_16_c1."node2")
     GROUP BY units
     ORDER BY count(graph_16_c1."node2") DESC
  PARAS: ['P2067']
---------------------------------------------
        3.88 real         3.05 user         0.22 sys

$kgtk query -i $WIKIDATA_PARTS/$quantity --graph-cache $STORE             -o $OUTPUT_FOLDER/Q11173.2.2.5.P2101.data_types.tsv             --match  '(n1)-[r:P21


[2020-10-20 11:50:45 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT kgtk_quantity_wd_units(graph_16_c1."node2") "units", count(graph_16_c1."node2") "value_count"
     FROM graph_16 AS graph_16_c1
     WHERE graph_16_c1."label"=?
     AND kgtk_quantity(graph_16_c1."node2")
     GROUP BY units
     ORDER BY count(graph_16_c1."node2") DESC
  PARAS: ['P2129']
---------------------------------------------
        0.61 real         0.47 user         0.11 sys

$kgtk query -i $WIKIDATA_PARTS/$quantity --graph-cache $STORE             -o $OUTPUT_FOLDER/Q11173.2.2.5.P2260.data_types.tsv             --match  '(n1)-[r:P2260]->(v)'             --return 'distinct kgtk_quantity_wd_units(v) as units, count(v) as value_count'             --where 'kgtk_quantity(v)'             --order-by 'count(v) desc' 

[2020-10-20 11:50:46 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT kgtk_quantity_wd_units(graph_16_c1."node2") "uni


[2020-10-20 11:50:52 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT kgtk_quantity_wd_units(graph_16_c1."node2") "units", count(graph_16_c1."node2") "value_count"
     FROM graph_16 AS graph_16_c1
     WHERE graph_16_c1."label"=?
     AND kgtk_quantity(graph_16_c1."node2")
     GROUP BY units
     ORDER BY count(graph_16_c1."node2") DESC
  PARAS: ['P2203']
---------------------------------------------
        0.58 real         0.46 user         0.11 sys

$kgtk query -i $WIKIDATA_PARTS/$quantity --graph-cache $STORE             -o $OUTPUT_FOLDER/Q11173.2.2.5.P1109.data_types.tsv             --match  '(n1)-[r:P1109]->(v)'             --return 'distinct kgtk_quantity_wd_units(v) as units, count(v) as value_count'             --where 'kgtk_quantity(v)'             --order-by 'count(v) desc' 

[2020-10-20 11:50:53 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT kgtk_quantity_wd_units(graph_16_c1."node2") "uni


[2020-10-20 11:50:59 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT kgtk_quantity_wd_units(graph_16_c1."node2") "units", count(graph_16_c1."node2") "value_count"
     FROM graph_16 AS graph_16_c1
     WHERE graph_16_c1."label"=?
     AND kgtk_quantity(graph_16_c1."node2")
     GROUP BY units
     ORDER BY count(graph_16_c1."node2") DESC
  PARAS: ['P2160']
---------------------------------------------
        0.62 real         0.48 user         0.11 sys

$kgtk query -i $WIKIDATA_PARTS/$quantity --graph-cache $STORE             -o $OUTPUT_FOLDER/Q11173.2.2.5.P2201.data_types.tsv             --match  '(n1)-[r:P2201]->(v)'             --return 'distinct kgtk_quantity_wd_units(v) as units, count(v) as value_count'             --where 'kgtk_quantity(v)'             --order-by 'count(v) desc' 

[2020-10-20 11:51:00 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT kgtk_quantity_wd_units(graph_16_c1."node2") "uni


[2020-10-20 11:51:05 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT kgtk_quantity_wd_units(graph_16_c1."node2") "units", count(graph_16_c1."node2") "value_count"
     FROM graph_16 AS graph_16_c1
     WHERE graph_16_c1."label"=?
     AND kgtk_quantity(graph_16_c1."node2")
     GROUP BY units
     ORDER BY count(graph_16_c1."node2") DESC
  PARAS: ['P2056']
---------------------------------------------
        0.58 real         0.46 user         0.10 sys

$kgtk query -i $WIKIDATA_PARTS/$quantity --graph-cache $STORE             -o $OUTPUT_FOLDER/Q11173.2.2.5.P2231.data_types.tsv             --match  '(n1)-[r:P2231]->(v)'             --return 'distinct kgtk_quantity_wd_units(v) as units, count(v) as value_count'             --where 'kgtk_quantity(v)'             --order-by 'count(v) desc' 

[2020-10-20 11:51:06 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT kgtk_quantity_wd_units(graph_16_c1."node2") "uni


[2020-10-20 11:51:12 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT kgtk_quantity_wd_units(graph_16_c1."node2") "units", count(graph_16_c1."node2") "value_count"
     FROM graph_16 AS graph_16_c1
     WHERE graph_16_c1."label"=?
     AND kgtk_quantity(graph_16_c1."node2")
     GROUP BY units
     ORDER BY count(graph_16_c1."node2") DESC
  PARAS: ['P1088']
---------------------------------------------
        0.82 real         0.60 user         0.16 sys

$kgtk query -i $WIKIDATA_PARTS/$quantity --graph-cache $STORE             -o $OUTPUT_FOLDER/Q11173.2.2.5.P2406.data_types.tsv             --match  '(n1)-[r:P2406]->(v)'             --return 'distinct kgtk_quantity_wd_units(v) as units, count(v) as value_count'             --where 'kgtk_quantity(v)'             --order-by 'count(v) desc' 

[2020-10-20 11:51:13 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT kgtk_quantity_wd_units(graph_16_c1."node2") "uni


[2020-10-20 11:51:18 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT kgtk_quantity_wd_units(graph_16_c1."node2") "units", count(graph_16_c1."node2") "value_count"
     FROM graph_16 AS graph_16_c1
     WHERE graph_16_c1."label"=?
     AND kgtk_quantity(graph_16_c1."node2")
     GROUP BY units
     ORDER BY count(graph_16_c1."node2") DESC
  PARAS: ['P5929']
---------------------------------------------
        0.58 real         0.45 user         0.11 sys

$kgtk query -i $WIKIDATA_PARTS/$quantity --graph-cache $STORE             -o $OUTPUT_FOLDER/Q11173.2.2.5.P6076.data_types.tsv             --match  '(n1)-[r:P6076]->(v)'             --return 'distinct kgtk_quantity_wd_units(v) as units, count(v) as value_count'             --where 'kgtk_quantity(v)'             --order-by 'count(v) desc' 

[2020-10-20 11:51:19 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT kgtk_quantity_wd_units(graph_16_c1."node2") "uni

## The file is loaded in the data frame and along with that I have also displayed the statistics for 'value_count'

In [44]:
for ele in df_quantity_data_type:
    display(ele)
    find_statistics(ele,"value_count")

,units,value_count
0,Q483261,146073
1,Q28924752,457
2,Q14623804,1


Statistics for : value_count
Distinct : 3
Distinct(%) :100.0%
Missing : 0
Missing(%) : 0.0%
Infinite : 0
Infinite(%) : 0.0%
Mean : 48843.666666666664
Max : 146073
Min : 1
Zeros : 0
Zeros(%) : 100.0%


,units,value_count
0,Q25267,8975
1,Q42289,506
2,Q11579,10
3,NaN,3


Statistics for : value_count
Distinct : 4
Distinct(%) :100.0%
Missing : 0
Missing(%) : 0.0%
Infinite : 0
Infinite(%) : 0.0%
Mean : 2373.5
Max : 8975
Min : 3
Zeros : 0
Zeros(%) : 100.0%


,units,value_count
0,Q13147228,1095
1,Q844211,8
2,Q834105,7
3,Q21604951,3


...

Statistics for : value_count
Distinct : 1
Distinct(%) :100.0%
Missing : 0
Missing(%) : 0.0%
Infinite : 0
Infinite(%) : 0.0%
Mean : 1.0
Max : 1
Min : 1
Zeros : 0
Zeros(%) : 100.0%


,units,value_count
0,Q11229,1


Statistics for : value_count
Distinct : 1
Distinct(%) :100.0%
Missing : 0
Missing(%) : 0.0%
Infinite : 0
Infinite(%) : 0.0%
Mean : 1.0
Max : 1
Min : 1
Zeros : 0
Zeros(%) : 100.0%


,units,value_count
0,Q11229,1


Statistics for : value_count
Distinct : 1
Distinct(%) :100.0%
Missing : 0
Missing(%) : 0.0%
Infinite : 0
Infinite(%) : 0.0%
Mean : 1.0
Max : 1
Min : 1
Zeros : 0
Zeros(%) : 100.0%


## In this cell we are finding the distribution of different range of magnitudes.

In [45]:
df_quantity_data_type_magnitude = []
try:
    df = pd.read_csv(os.path.join(os.getenv('OUTPUT_FOLDER'),os.getenv('property_summary_quantity')),delimiter='\t')
    cmd = "$kgtk query -i $WIKIDATA_PARTS/$quantity --graph-cache $STORE \
            -o $OUTPUT_FOLDER/__output \
            --match  '(n1)-[r:__property]->(v)' \
            --return 'distinct kgtk_quantity_number_int(v) as magnitude, count(v) as value_count' \
            --where 'kgtk_quantity(v)' \
            --order-by 'count(v) desc' "
    for index, ele in df.iterrows():
        output_file = subset_name + ".2.2.5." + ele["property"]+".data_types_magnitude"+".tsv"
        run_command(cmd, {"__property": ele["property"],"__output": output_file})
        temp = pd.read_csv(os.path.join(os.getenv('OUTPUT_FOLDER'),output_file),delimiter='\t')
        temp['magnitude'] = pd.cut(temp['magnitude'], bins=10)
        temp = temp['magnitude'].value_counts().rename_axis('magnitude').reset_index(name='counts')
        df_quantity_data_type_magnitude.append(temp)
        temp.to_csv(os.path.join(os.getenv('OUTPUT_FOLDER'),output_file),sep='\t',index=False)
except Exception as e:
    print(e)

$kgtk query -i $WIKIDATA_PARTS/$quantity --graph-cache $STORE             -o $OUTPUT_FOLDER/Q11173.2.2.5.P2067.data_types_magnitude.tsv             --match  '(n1)-[r:P2067]->(v)'             --return 'distinct kgtk_quantity_number_int(v) as magnitude, count(v) as value_count'             --where 'kgtk_quantity(v)'             --order-by 'count(v) desc' 

[2020-10-20 11:53:55 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT kgtk_quantity_number_int(graph_16_c1."node2") "magnitude", count(graph_16_c1."node2") "value_count"
     FROM graph_16 AS graph_16_c1
     WHERE graph_16_c1."label"=?
     AND kgtk_quantity(graph_16_c1."node2")
     GROUP BY magnitude
     ORDER BY count(graph_16_c1."node2") DESC
  PARAS: ['P2067']
---------------------------------------------
        2.65 real         2.46 user         0.14 sys

$kgtk query -i $WIKIDATA_PARTS/$quantity --graph-cache $STORE             -o $OUTPUT_FOLDER/Q11173.2.2.5.P2101.data_types_magnitude.t


[2020-10-20 11:54:04 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT kgtk_quantity_number_int(graph_16_c1."node2") "magnitude", count(graph_16_c1."node2") "value_count"
     FROM graph_16 AS graph_16_c1
     WHERE graph_16_c1."label"=?
     AND kgtk_quantity(graph_16_c1."node2")
     GROUP BY magnitude
     ORDER BY count(graph_16_c1."node2") DESC
  PARAS: ['P2129']
---------------------------------------------
        0.69 real         0.52 user         0.13 sys

$kgtk query -i $WIKIDATA_PARTS/$quantity --graph-cache $STORE             -o $OUTPUT_FOLDER/Q11173.2.2.5.P2260.data_types_magnitude.tsv             --match  '(n1)-[r:P2260]->(v)'             --return 'distinct kgtk_quantity_number_int(v) as magnitude, count(v) as value_count'             --where 'kgtk_quantity(v)'             --order-by 'count(v) desc' 

[2020-10-20 11:54:05 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT kgtk_quantity_number_i

## In the below cell, all the computed files are loaded in the data frame and also the statistics of the 'count' is printed.

In [46]:
for ele in df_quantity_data_type_magnitude:
    display(ele)
    find_statistics(ele,"counts")

,magnitude,counts
0,"(-743.511, 74553.1]",2130
1,"(298206.4, 372757.5]",5
2,"(223655.3, 298206.4]",3
3,"(149104.2, 223655.3]",3
4,"(74553.1, 149104.2]",2
5,"(670961.9, 745513.0]",1
6,"(521859.7, 596410.8]",1
7,"(447308.6, 521859.7]",1
8,"(596410.8, 670961.9]",0
9,"(372757.5, 447308.6]",0


Statistics for : counts
Distinct : 6
Distinct(%) :60.0%
Missing : 0
Missing(%) : 0.0%
Infinite : 0
Infinite(%) : 0.0%
Mean : 214.6
Max : 2130
Min : 0
Zeros : 2
Zeros(%) : 60.0%


,magnitude,counts
0,"(-385.029, 1031.9]",660
1,"(1031.9, 2434.8]",79
2,"(2434.8, 3837.7]",16
3,"(3837.7, 5240.6]",3
4,"(12255.1, 13658.0]",1
5,"(10852.2, 12255.1]",0
6,"(9449.3, 10852.2]",0
7,"(8046.4, 9449.3]",0
8,"(6643.5, 8046.4]",0
9,"(5240.6, 6643.5]",0


Statistics for : counts
Distinct : 6
Distinct(%) :60.0%
Missing : 0
Missing(%) : 0.0%
Infinite : 0
Infinite(%) : 0.0%
Mean : 75.9
Max : 660
Min : 0
Zeros : 5
Zeros(%) : 60.0%


,magnitude,counts
0,"(-1.9, 190.0]",15
1,"(760.0, 950.0]",5
2,"(1520.0, 1710.0]",2
3,"(1710.0, 1900.0]",1
4,"(1140.0, 1330.0]",1
5,"(1330.0, 1520.0]",0
6,"(950.0, 1140.0]",0
7,"(570.0, 760.0]",0
8,"(380.0, 570.0]",0
9,"(190.0, 380.0]",0


...

Statistics for : counts
Distinct : 4
Distinct(%) :40.0%
Missing : 0
Missing(%) : 0.0%
Infinite : 0
Infinite(%) : 0.0%
Mean : 10.9
Max : 104
Min : 0
Zeros : 6
Zeros(%) : 40.0%


,magnitude,counts
0,"(293.0, 614.0]",50
1,"(-31.21, 293.0]",50
2,"(614.0, 935.0]",18
3,"(935.0, 1256.0]",8
4,"(2861.0, 3182.0]",4
5,"(1577.0, 1898.0]",3
6,"(1898.0, 2219.0]",2
7,"(1256.0, 1577.0]",2
8,"(2540.0, 2861.0]",1
9,"(2219.0, 2540.0]",1


Statistics for : counts
Distinct : 7
Distinct(%) :70.0%
Missing : 0
Missing(%) : 0.0%
Infinite : 0
Infinite(%) : 0.0%
Mean : 13.9
Max : 50
Min : 1
Zeros : 0
Zeros(%) : 70.0%


,magnitude,counts
0,"(-5.6, 560.0]",36
1,"(1680.0, 2240.0]",3
2,"(560.0, 1120.0]",2
3,"(5040.0, 5600.0]",1
4,"(2240.0, 2800.0]",1
5,"(1120.0, 1680.0]",1
6,"(4480.0, 5040.0]",0
7,"(3920.0, 4480.0]",0
8,"(3360.0, 3920.0]",0
9,"(2800.0, 3360.0]",0


Statistics for : counts
Distinct : 5
Distinct(%) :50.0%
Missing : 0
Missing(%) : 0.0%
Infinite : 0
Infinite(%) : 0.0%
Mean : 4.4
Max : 36
Min : 0
Zeros : 4
Zeros(%) : 50.0%


## In the below cell, distribution of dates for top K properties is being found and also finding the statistics for the 'counts' column

In [47]:
df_time_data_type_year = []
try:
    df = pd.read_csv(os.path.join(os.getenv('OUTPUT_FOLDER'),os.getenv('property_summary_time')),delimiter='\t')
    cmd = "$kgtk query -i $WIKIDATA_PARTS/$time --graph-cache $STORE \
            -o $OUTPUT_FOLDER/__output \
            --match  '(n1)-[r:__property]->(v)' \
            --return 'distinct kgtk_date_year(v) as year, count(v) as value_count' \
            --where 'kgtk_date(v)' \
            --order-by 'count(v) desc' "
    for index, ele in df.iterrows():
        output_file = subset_name + ".2.2.2." + ele["property"]+".data_types_year"+".tsv"
        run_command(cmd, {"__property": ele["property"],"__output": output_file})
        temp = pd.read_csv(os.path.join(os.getenv('OUTPUT_FOLDER'),output_file),delimiter='\t')
        df_time_data_type_year.append(temp)
except Exception as e:
    print(e)

$kgtk query -i $WIKIDATA_PARTS/$time --graph-cache $STORE             -o $OUTPUT_FOLDER/Q11173.2.2.2.P575.data_types_year.tsv             --match  '(n1)-[r:P575]->(v)'             --return 'distinct kgtk_date_year(v) as year, count(v) as value_count'             --where 'kgtk_date(v)'             --order-by 'count(v) desc' 

[2020-10-20 11:54:51 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT kgtk_date_year(graph_17_c1."node2") "year", count(graph_17_c1."node2") "value_count"
     FROM graph_17 AS graph_17_c1
     WHERE graph_17_c1."label"=?
     AND kgtk_date(graph_17_c1."node2")
     GROUP BY year
     ORDER BY count(graph_17_c1."node2") DESC
  PARAS: ['P575']
---------------------------------------------
        0.59 real         0.46 user         0.11 sys

$kgtk query -i $WIKIDATA_PARTS/$time --graph-cache $STORE             -o $OUTPUT_FOLDER/Q11173.2.2.2.P2669.data_types_year.tsv             --match  '(n1)-[r:P2669]->(v)'             --retu

## In the below cell the each of the above generated table is loaded in the dataframe and also printing the statistics for the 'value_count' column

In [48]:
for ele in df_time_data_type_year:
    display(ele.head())
    find_statistics(ele,'value_count')

,year,value_count
0,1856,3
1,1847,2
2,1993,1
3,1984,1
4,1970,1


Statistics for : value_count
Distinct : 3
Distinct(%) :17.65%
Missing : 0
Missing(%) : 0.0%
Infinite : 0
Infinite(%) : 0.0%
Mean : 1.1764705882352942
Max : 3
Min : 1
Zeros : 0
Zeros(%) : 17.65%


,year,value_count
0,2013,1


Statistics for : value_count
Distinct : 1
Distinct(%) :100.0%
Missing : 0
Missing(%) : 0.0%
Infinite : 0
Infinite(%) : 0.0%
Mean : 1.0
Max : 1
Min : 1
Zeros : 0
Zeros(%) : 100.0%


,year,value_count
0,1933,1


Statistics for : value_count
Distinct : 1
Distinct(%) :100.0%
Missing : 0
Missing(%) : 0.0%
Infinite : 0
Infinite(%) : 0.0%
Mean : 1.0
Max : 1
Min : 1
Zeros : 0
Zeros(%) : 100.0%


,year,value_count
0,1961,1


Statistics for : value_count
Distinct : 1
Distinct(%) :100.0%
Missing : 0
Missing(%) : 0.0%
Infinite : 0
Infinite(%) : 0.0%
Mean : 1.0
Max : 1
Min : 1
Zeros : 0
Zeros(%) : 100.0%


,year,value_count
0,1970,1


Statistics for : value_count
Distinct : 1
Distinct(%) :100.0%
Missing : 0
Missing(%) : 0.0%
Infinite : 0
Infinite(%) : 0.0%
Mean : 1.0
Max : 1
Min : 1
Zeros : 0
Zeros(%) : 100.0%


## In the below cell the distribution time based on time zone is generated for top K properties.

In [49]:
df_time_data_type_zone = []
try:
    df = pd.read_csv(os.path.join(os.getenv('OUTPUT_FOLDER'),os.getenv('property_summary_time')),delimiter='\t')
    cmd = "$kgtk query -i $WIKIDATA_PARTS/$time --graph-cache $STORE \
            -o $OUTPUT_FOLDER/__output \
            --match  '(n1)-[r:__property]->(v)' \
            --return 'distinct kgtk_date_zone_string(v) as zone, count(v) as value_count' \
            --where 'kgtk_date(v)' \
            --order-by 'count(v) desc'"
    for index, ele in df.iterrows():
        output_file = subset_name + ".2.2.2." + ele["property"]+".data_types_zone"+".tsv"
        run_command(cmd, {"__property": ele["property"],"__output": output_file})
        temp = pd.read_csv(os.path.join(os.getenv('OUTPUT_FOLDER'),output_file),delimiter='\t')
        df_time_data_type_zone.append(temp)
except Exception as e:
    print(e)

$kgtk query -i $WIKIDATA_PARTS/$time --graph-cache $STORE             -o $OUTPUT_FOLDER/Q11173.2.2.2.P575.data_types_zone.tsv             --match  '(n1)-[r:P575]->(v)'             --return 'distinct kgtk_date_zone_string(v) as zone, count(v) as value_count'             --where 'kgtk_date(v)'             --order-by 'count(v) desc'

[2020-10-20 11:55:46 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT kgtk_date_zone_string(graph_17_c1."node2") "zone", count(graph_17_c1."node2") "value_count"
     FROM graph_17 AS graph_17_c1
     WHERE graph_17_c1."label"=?
     AND kgtk_date(graph_17_c1."node2")
     GROUP BY zone
     ORDER BY count(graph_17_c1."node2") DESC
  PARAS: ['P575']
---------------------------------------------
        0.58 real         0.45 user         0.10 sys

$kgtk query -i $WIKIDATA_PARTS/$time --graph-cache $STORE             -o $OUTPUT_FOLDER/Q11173.2.2.2.P2669.data_types_zone.tsv             --match  '(n1)-[r:P2669]->(v)'      

## The file is loaded in the data frame and along with that I have also displayed the statistics for value_count

In [50]:
for ele in df_time_data_type_zone:
    display(ele)
    find_statistics(ele,'value_count')

,zone,value_count
0,Z,20


Statistics for : value_count
Distinct : 1
Distinct(%) :100.0%
Missing : 0
Missing(%) : 0.0%
Infinite : 0
Infinite(%) : 0.0%
Mean : 20.0
Max : 20
Min : 20
Zeros : 0
Zeros(%) : 100.0%


,zone,value_count
0,Z,1


Statistics for : value_count
Distinct : 1
Distinct(%) :100.0%
Missing : 0
Missing(%) : 0.0%
Infinite : 0
Infinite(%) : 0.0%
Mean : 1.0
Max : 1
Min : 1
Zeros : 0
Zeros(%) : 100.0%


,zone,value_count
0,Z,1


Statistics for : value_count
Distinct : 1
Distinct(%) :100.0%
Missing : 0
Missing(%) : 0.0%
Infinite : 0
Infinite(%) : 0.0%
Mean : 1.0
Max : 1
Min : 1
Zeros : 0
Zeros(%) : 100.0%


,zone,value_count
0,Z,1


Statistics for : value_count
Distinct : 1
Distinct(%) :100.0%
Missing : 0
Missing(%) : 0.0%
Infinite : 0
Infinite(%) : 0.0%
Mean : 1.0
Max : 1
Min : 1
Zeros : 0
Zeros(%) : 100.0%


,zone,value_count
0,Z,1


Statistics for : value_count
Distinct : 1
Distinct(%) :100.0%
Missing : 0
Missing(%) : 0.0%
Infinite : 0
Infinite(%) : 0.0%
Mean : 1.0
Max : 1
Min : 1
Zeros : 0
Zeros(%) : 100.0%


In [51]:
df_string_data_type_lang = []
try:
    df = pd.read_csv(os.path.join(os.getenv('OUTPUT_FOLDER'),os.getenv('property_summary_string')),delimiter='\t')
    cmd = "$kgtk query -i $WIKIDATA_PARTS/$string --graph-cache $STORE \
            -o $OUTPUT_FOLDER/__output \
            --match  '(n1)-[r:__property]->(v)' \
            --return 'distinct kgtk_lqstring_lang(v) as language, count(v) as value_count' \
            --where 'kgtk_lqstring(v)' \
            --order-by 'count(v) desc'"
    for index, ele in df.iterrows():
        output_file = subset_name + ".2.2.13." + ele["property"]+".data_types_lang"+".tsv"
        run_command(cmd, {"__property": ele["property"],"__output": output_file})
        temp = pd.read_csv(os.path.join(os.getenv('OUTPUT_FOLDER'),output_file),delimiter='\t')
        df_string_data_type_lang.append(temp)
except Exception as e:
    print(e)

$kgtk query -i $WIKIDATA_PARTS/$string --graph-cache $STORE             -o $OUTPUT_FOLDER/Q11173.2.2.13.P274.data_types_lang.tsv             --match  '(n1)-[r:P274]->(v)'             --return 'distinct kgtk_lqstring_lang(v) as language, count(v) as value_count'             --where 'kgtk_lqstring(v)'             --order-by 'count(v) desc'

[2020-10-20 11:56:20 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT kgtk_lqstring_lang(graph_18_c1."node2") "language", count(graph_18_c1."node2") "value_count"
     FROM graph_18 AS graph_18_c1
     WHERE graph_18_c1."label"=?
     AND kgtk_lqstring(graph_18_c1."node2")
     GROUP BY language
     ORDER BY count(graph_18_c1."node2") DESC
  PARAS: ['P274']
---------------------------------------------
        1.90 real         1.13 user         0.28 sys

$kgtk query -i $WIKIDATA_PARTS/$string --graph-cache $STORE             -o $OUTPUT_FOLDER/Q11173.2.2.13.P233.data_types_lang.tsv             --match  '(n1)-[r


[2020-10-20 11:56:30 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT kgtk_lqstring_lang(graph_18_c1."node2") "language", count(graph_18_c1."node2") "value_count"
     FROM graph_18 AS graph_18_c1
     WHERE graph_18_c1."label"=?
     AND kgtk_lqstring(graph_18_c1."node2")
     GROUP BY language
     ORDER BY count(graph_18_c1."node2") DESC
  PARAS: ['P591']
---------------------------------------------
        0.73 real         0.55 user         0.13 sys

$kgtk query -i $WIKIDATA_PARTS/$string --graph-cache $STORE             -o $OUTPUT_FOLDER/Q11173.2.2.13.P874.data_types_lang.tsv             --match  '(n1)-[r:P874]->(v)'             --return 'distinct kgtk_lqstring_lang(v) as language, count(v) as value_count'             --where 'kgtk_lqstring(v)'             --order-by 'count(v) desc'

[2020-10-20 11:56:31 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT kgtk_lqstring_lang(graph_18_c1."node2") "langua


[2020-10-20 11:56:37 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT kgtk_lqstring_lang(graph_18_c1."node2") "language", count(graph_18_c1."node2") "value_count"
     FROM graph_18 AS graph_18_c1
     WHERE graph_18_c1."label"=?
     AND kgtk_lqstring(graph_18_c1."node2")
     GROUP BY language
     ORDER BY count(graph_18_c1."node2") DESC
  PARAS: ['P593']
---------------------------------------------
        0.64 real         0.48 user         0.12 sys

$kgtk query -i $WIKIDATA_PARTS/$string --graph-cache $STORE             -o $OUTPUT_FOLDER/Q11173.2.2.13.P875.data_types_lang.tsv             --match  '(n1)-[r:P875]->(v)'             --return 'distinct kgtk_lqstring_lang(v) as language, count(v) as value_count'             --where 'kgtk_lqstring(v)'             --order-by 'count(v) desc'

[2020-10-20 11:56:38 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT kgtk_lqstring_lang(graph_18_c1."node2") "langua


[2020-10-20 11:56:44 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT kgtk_lqstring_lang(graph_18_c1."node2") "language", count(graph_18_c1."node2") "value_count"
     FROM graph_18 AS graph_18_c1
     WHERE graph_18_c1."label"=?
     AND kgtk_lqstring(graph_18_c1."node2")
     GROUP BY language
     ORDER BY count(graph_18_c1."node2") DESC
  PARAS: ['P246']
---------------------------------------------
        0.61 real         0.47 user         0.11 sys

$kgtk query -i $WIKIDATA_PARTS/$string --graph-cache $STORE             -o $OUTPUT_FOLDER/Q11173.2.2.13.P2572.data_types_lang.tsv             --match  '(n1)-[r:P2572]->(v)'             --return 'distinct kgtk_lqstring_lang(v) as language, count(v) as value_count'             --where 'kgtk_lqstring(v)'             --order-by 'count(v) desc'

[2020-10-20 11:56:44 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT kgtk_lqstring_lang(graph_18_c1."node2") "lang

In [ ]:
for ele in df_string_data_type_lang:
    display(ele)
    ele.describe()

## 2.3 Item properties
### Top K qnodes used in node2
### qnode, label, count

## Header

### qnode -- qnode corresponding to node2 
### count -- number of times qnode is present 
### label -- label of the qnode



In [54]:
cmd  = "$kgtk query  -i $WIKIDATA_PARTS/$wikibase_item -i $WIKIDATA_PARTS/$label --graph-cache $STORE \
-o $OUTPUT_FOLDER/$item_properties \
--match 'part: (n1)-[l]->(n2), label: (n2)-[:label]->(label)' \
--return 'distinct n2 as qnode, count(n2) as count, label ' \
--where '(label.kgtk_lqstring_lang_suffix = \"en\") AND (n2 != \"__subset_name\")' \
--order-by 'count(n2) desc'"
run_command(cmd, {"__subset_name":subset_name})

$kgtk query  -i $WIKIDATA_PARTS/$wikibase_item -i $WIKIDATA_PARTS/$label --graph-cache $STORE -o $OUTPUT_FOLDER/$item_properties --match 'part: (n1)-[l]->(n2), label: (n2)-[:label]->(label)' --return 'distinct n2 as qnode, count(n2) as count, label ' --where '(label.kgtk_lqstring_lang_suffix = "en") AND (n2 != "Q11173")' --order-by 'count(n2) desc'

[2020-10-20 11:57:27 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_20_c2."node1" "qnode", count(graph_20_c2."node1") "count", graph_20_c2."node2"
     FROM graph_19 AS graph_19_c1, graph_20 AS graph_20_c2
     WHERE graph_20_c2."label"=?
     AND graph_19_c1."node2"=graph_20_c2."node1"
     AND ((kgtk_lqstring_lang_suffix(graph_20_c2."node2") = ?) AND (graph_20_c2."node1" != ?))
     GROUP BY qnode
     ORDER BY count(graph_20_c2."node1") DESC
  PARAS: ['label', 'en', 'Q11173']
---------------------------------------------
        3.63 real         3.06 user         0.31 sys



## The file is loaded in the data frame and along with that I have also displayed the statistics for 'count' column

In [56]:
df_item = pd.read_csv(os.path.join(os.getenv('OUTPUT_FOLDER'),os.getenv('item_properties')),delimiter='\t')
display(df_item.head())
find_statistics(df_item,"count")

,qnode,count,node2
0,Q90481889,49433,'CAS COVID-19 Anti-Viral Candidate Compounds'@en
1,Q623,6174,'carbon'@en
2,Q629,5434,'oxygen'@en
3,Q12140,5048,'medication'@en
4,Q627,2418,'nitrogen'@en


Statistics for : count
Distinct : 191
Distinct(%) :1.15%
Missing : 0
Missing(%) : 0.0%
Infinite : 0
Infinite(%) : 0.0%
Mean : 11.902906208718626
Max : 49433
Min : 1
Zeros : 0
Zeros(%) : 1.15%


## 2.4 Time properties
### min time, max time
### Chart: x axis is different times, y axis is count of nodes that have the value. Binning maybe required


In [51]:
!$kgtk query  -i $WIKIDATA_PARTS/$time  --graph-cache $STORE \
-o $OUTPUT_FOLDER/$time_properties \
--match 'part: (n1)-[l]->(time)' \
--return 'min(time) as min_time, max(time) as max_time'

[2020-10-13 13:01:52 query]: SQL Translation:
---------------------------------------------
  SELECT min(graph_5_c1."node2") "min_time", max(graph_5_c1."node2") "max_time"
     FROM graph_5 AS graph_5_c1
  PARAS: []
---------------------------------------------
min_time	max_time
^1669-00-00T00:00:00Z/9	^2013-05-00T00:00:00Z/10
        0.55 real         0.43 user         0.11 sys


## Count of top K distinct time 

## Header

### time -- time
### count --- number of times it is present



In [57]:
!$kgtk query  -i $WIKIDATA_PARTS/$time  --graph-cache $STORE \
-o $OUTPUT_FOLDER/$time_properties_count \
--match 'part: (n1)-[l]->(time)' \
--return 'distinct time as `time`, count(time) as count' \
--order-by 'count(time) desc'

[2020-10-20 11:58:21 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_17_c1."node2" "time", count(graph_17_c1."node2") "count"
     FROM graph_17 AS graph_17_c1
     GROUP BY time
     ORDER BY count(graph_17_c1."node2") DESC
  PARAS: []
---------------------------------------------
        0.59 real         0.44 user         0.11 sys


## The file is loaded in the data frame and along with that I have also displayed the statistics

In [59]:
df_time = pd.read_csv(os.path.join(os.getenv('OUTPUT_FOLDER'),os.getenv('time_properties_count')),delimiter='\t')
display(df_time.head())
df_time.describe()
find_statistics(df_time,"count")

,time,count
0,^1856-00-00T00:00:00Z/9,2
1,^1847-00-00T00:00:00Z/9,2
2,^2013-05-00T00:00:00Z/10,1
3,^1993-00-00T00:00:00Z/9,1
4,^1987-00-00T00:00:00Z/9,1


Statistics for : count
Distinct : 2
Distinct(%) :8.7%
Missing : 0
Missing(%) : 0.0%
Infinite : 0
Infinite(%) : 0.0%
Mean : 1.0869565217391304
Max : 2
Min : 1
Zeros : 0
Zeros(%) : 8.7%


## 2.5 Quantity properties
### list of units and counts
### Chart: x axis is different magnitudes, y axis is count of nodes that have the value. Binning is required

## Header
### units -- units
### counts -- count corresponding to each unit


In [60]:
!$kgtk query  -i $WIKIDATA_PARTS/$quantity -i $WIKIDATA_PARTS/$label --graph-cache $STORE \
-o $OUTPUT_FOLDER/$quantity_properties \
--match 'part: (n1)-[l{label: llab}]->(n2), label: (llab)-[:label]->(label)' \
--return 'distinct label as units, count(llab) as counts'\
--where 'label.kgtk_lqstring_lang_suffix = "en"' \
--order-by 'count(llab) desc'

[2020-10-20 11:59:27 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_20_c2."node2" "units", count(graph_16_c1."label") "counts"
     FROM graph_16 AS graph_16_c1, graph_20 AS graph_20_c2
     WHERE graph_16_c1."label"=graph_16_c1."label"
     AND graph_20_c2."label"=?
     AND graph_16_c1."label"=graph_20_c2."node1"
     AND (kgtk_lqstring_lang_suffix(graph_20_c2."node2") = ?)
     GROUP BY units
     ORDER BY count(graph_16_c1."label") DESC
  PARAS: ['label', 'en']
---------------------------------------------
        1.60 real         1.42 user         0.13 sys


## The file is loaded in the data frame and along with that I have also displayed the statistics for counts column

In [61]:
df_quantity = pd.read_csv(os.path.join(os.getenv('OUTPUT_FOLDER'),os.getenv('quantity_properties')),delimiter='\t')
display(df_quantity.head())
find_statistics(df_quantity,"counts")

,units,counts
0,'mass'@en,146546
1,'melting point'@en,9494
2,'density'@en,1113
3,'defined daily dose'@en,908
4,'boiling point'@en,873


Statistics for : counts
Distinct : 42
Distinct(%) :67.74%
Missing : 0
Missing(%) : 0.0%
Infinite : 0
Infinite(%) : 0.0%
Mean : 2661.5967741935483
Max : 146546
Min : 1
Zeros : 0
Zeros(%) : 67.74%


## 2.6 String, Monolingual, url, External id, Math musical notation
### number of distinct values
### list of top M values

## Header
### value -- value
### number_of_statements -- number of statements corresponding to that value.



In [63]:
types = [
    ("math","distinct_values_math"),
    ("string","distinct_values_string"),
    ("external_id","distinct_values_external_id"),
    ("monolingualtext","distinct_values_monolingual_text"),
    ("musical_notation","distinct_values_musical_notation"),
]

cmd = "$kgtk query  -i $WIKIDATA_PARTS/$TYPE_FILE  --graph-cache $STORE \
-o $OUTPUT_FOLDER/$output_file \
--match 'part: (n1)-[l]->(n2)' \
--return 'distinct n2 as value, count(n2) as number_of_statements' \
--order-by 'count(n2) desc'"

for type, output_file in types:
    run_command(cmd, {"TYPE_FILE": type,"output_file":output_file})

$kgtk query  -i $WIKIDATA_PARTS/$math  --graph-cache $STORE -o $OUTPUT_FOLDER/$distinct_values_math --match 'part: (n1)-[l]->(n2)' --return 'distinct n2 as value, count(n2) as number_of_statements' --order-by 'count(n2) desc'

[2020-10-20 12:00:14 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_22_c1."node2" "value", count(graph_22_c1."node2") "number_of_statements"
     FROM graph_22 AS graph_22_c1
     GROUP BY value
     ORDER BY count(graph_22_c1."node2") DESC
  PARAS: []
---------------------------------------------
        0.57 real         0.44 user         0.11 sys

$kgtk query  -i $WIKIDATA_PARTS/$string  --graph-cache $STORE -o $OUTPUT_FOLDER/$distinct_values_string --match 'part: (n1)-[l]->(n2)' --return 'distinct n2 as value, count(n2) as number_of_statements' --order-by 'count(n2) desc'

[2020-10-20 12:00:15 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_18_c1."node2" "value", co

## I have loaded all the files in dataframe and also displayed the statistics for 'number_of_statemens' for each of the dataframe

In [64]:
types = [
    ("math","distinct_values_math"),
    ("string","distinct_values_string"),
    ("external_id","distinct_values_external_id"),
    ("monolingualtext","distinct_values_monolingual_text"),
    ("musical_notation","distinct_values_musical_notation"),
]
df_distinct_values = []
for type, output_file in types:
    try :
        temp = pd.read_csv(os.path.join(os.getenv('OUTPUT_FOLDER'),os.getenv(output_file)),delimiter='\t')
    except Exception as e:
        continue
    df_distinct_values.append(temp)
    display(df_distinct_values[-1].head())
    find_statistics(temp,"number_of_statements")

,value,number_of_statements


Statistics for : number_of_statements
Distinct : 0
Distinct(%) :nan%
Missing : 0
Missing(%) : nan%
Infinite : 0
Infinite(%) : nan%
Mean : nan
Max : nan
Min : nan
Zeros : 0
Zeros(%) : nan%


/Users/shashanksaurabh/.local/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in long_scalars
  """
/Users/shashanksaurabh/.local/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in long_scalars
  if __name__ == '__main__':
/Users/shashanksaurabh/.local/lib/python3.7/site-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in long_scalars
  del sys.path[0]
/Users/shashanksaurabh/.local/lib/python3.7/site-packages/ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in long_scalars


,value,number_of_statements
0,NaN,890
1,C₆₁H₁₀₆O₆,247
2,C₆₁H₁₀₄O₆,245
3,C₆₃H₁₁₀O₆,239
4,C₆₁H₁₀₂O₆,236


Statistics for : number_of_statements
Distinct : 162
Distinct(%) :0.04%
Missing : 0
Missing(%) : 0.0%
Infinite : 0
Infinite(%) : 0.0%
Mean : 1.5876895739493369
Max : 890
Min : 1
Zeros : 0
Zeros(%) : 0.04%


...

,value,number_of_statements
0,'zopiclone'@en,2
1,'tomoxetine'@en,2
2,'sulfamoxole'@en,2
3,'pembrolizumab'@en,2
4,'doxycycline'@en,2


Statistics for : number_of_statements
Distinct : 2
Distinct(%) :0.08%
Missing : 0
Missing(%) : 0.0%
Infinite : 0
Infinite(%) : 0.0%
Mean : 1.0061753808151503
Max : 2
Min : 1
Zeros : 0
Zeros(%) : 0.08%


,value,number_of_statements


Statistics for : number_of_statements
Distinct : 0
Distinct(%) :nan%
Missing : 0
Missing(%) : nan%
Infinite : 0
Infinite(%) : nan%
Mean : nan
Max : nan
Min : nan
Zeros : 0
Zeros(%) : nan%


/Users/shashanksaurabh/.local/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in long_scalars
  """
/Users/shashanksaurabh/.local/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in long_scalars
  if __name__ == '__main__':
/Users/shashanksaurabh/.local/lib/python3.7/site-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in long_scalars
  del sys.path[0]
/Users/shashanksaurabh/.local/lib/python3.7/site-packages/ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in long_scalars


## 2.7 Globe coordinate
### map with top M, put circles with radius proportional to the number of nodes that have the coordinate 

## Header
### Coordinate -- Coordinate
### Count -- Number of instances of the coordinate


In [67]:
!$kgtk query  -i $WIKIDATA_PARTS/$globe_coordinate -i $WIKIDATA_PARTS/$label --graph-cache $STORE \
-o $OUTPUT_FOLDER/$globe_coordinate_top_m \
--match 'part: (n1)-[l]->(n2)' \
--return 'distinct n2 as Coordinate, count(n2) as count'\
--order-by 'count(n2) desc'

[2020-10-20 12:03:08 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_26_c1."node2" "Coordinate", count(graph_26_c1."node2") "count"
     FROM graph_26 AS graph_26_c1
     GROUP BY Coordinate
     ORDER BY count(graph_26_c1."node2") DESC
  PARAS: []
---------------------------------------------
        0.59 real         0.45 user         0.11 sys


## I have loaded all the files in dataframe and also displayed the statistics for 'count' for each of the dataframe

In [69]:
df_coordinate = pd.read_csv(os.path.join(os.getenv('OUTPUT_FOLDER'),os.getenv('globe_coordinate_top_m')),delimiter='\t')
display(df_coordinate.head())
find_statistics(df_coordinate,"count")


,Coordinate,count
0,@37.25/27,1


Statistics for : count
Distinct : 1
Distinct(%) :100.0%
Missing : 0
Missing(%) : 0.0%
Infinite : 0
Infinite(%) : 0.0%
Mean : 1.0
Max : 1
Min : 1
Zeros : 0
Zeros(%) : 100.0%


## 2.8 geo-shape
### random sample of M nodes that have the property

## Header
### node1 -- node1
### label -- label
### node2 -- node2


In [80]:
try:
    df = pd.read_csv(os.path.join(os.getenv('WIKIDATA_PARTS'),os.getenv('geo_shape')),delimiter='\t',index_col=False)
    try:
        num_rows = min(int(K[1:-1]),len(df))
    except Exception as e:
        num_rows = min(int(K),len(df))
    df.sample(n=num_rows)
    display(df)
    df.to_csv(os.path.join(os.getenv('OUTPUT_FOLDER'),os.getenv('geo_shape_top_m')),sep='\t',index=False)
except Exception as e:
    print(e)

,id,node1,label,node2
